In [2]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from scipy.signal import savgol_filter as sg
from scipy.signal import find_peaks as fp
from scipy.optimize import curve_fit as lsqFit
from operator import itemgetter
from ipywidgets import Output, VBox, HBox
from ipywidgets.embed import embed_minimal_html as html


'''
Opens a txt file output from labspec6 and reads the spectral data into the program.
Takes in the file path.
isMap, x, y (optional): tells the function whether the data to be imported is a map. If it is then the x and y range of the map must be specified for the data to import correctly.
Returns two arrays, one containing the wavenumber range (x-axis), and one containing intensity (y-axis)
'''
def importSpectrum(path, isMap = False, x = 1, y = 1):
    
    #code for importing maps
    if isMap:
        
        #open file and read the wavenumber data from the first line
        f = open(path)
        wavenumberRange = np.array(f.readline().split(), dtype = np.float32)
        
        #create empty array to hold the data
        grid = np.empty([x,y,len(wavenumberRange)])
        
        #iterate through the file based on map grid size
        for i in range(x):
            for j in range(y):
                
                #read each line and split it into a list 
                temp = f.readline().split()
                
                #remove the first two entries (coordinates) from the list, then add it to the grid
                grid[i,j] = np.array(temp[2:], dtype = np.float32)
        
        #close the file and return the data
        f.close()
        return(wavenumberRange, grid)
    
    #code for importing single spectra
    else:
        
        #open the file and read each line into a list
        with open(path) as f:
            rawData = []
            for line in f:
                rawData.append(line.split())
        
        #convert list into an array and then transpose it to switch from sets of coordinates to seperate x and y data
        rawData = np.array(rawData, dtype = np.float32)
        data = np.transpose(rawData)
        
        #close the file and return the data
        f.close()
        return(data[0], data[1])

'''
Creates a figure display a set of plots using the same x-axis.
Takes in a set of x points and a list of sets of y points.
Outputs a plotly figure, no returns.
'''
def createFig(x, Y):
    
    #create figure
    fig = go.Figure()
    
    #for each set of y data add a scatter plot to the figure
    for y in Y:
        fig.add_trace(go.Scatter(x = x, y = y))
    
    #create and open html file containing the figure
    fig.write_html('spectrum.html', auto_open=True)

'''
Compute the derivative of a set of x-y data using a simple central difference formula.
Numpy's more complex gradient function seems to suppress the presence of the raman peaks in the derivative (possibly due to the noisy data), so this custom function is required.
Takes in a set of x data and a set of y data, both sets must be of equal length.
Returns the derivative dy/dx.
'''
def derivxy(x, y):

    #create empty array to hold derivative
    dy = np.zeros(len(y))
    
    #compute boundary values using forward diffrence and backwards difference
    dy[0] = (y[1] - y[0])/(x[1] - x[0])
    dy[-1] = (y[-1] - y[-2])/(x[-1] - x[-2])
    
    #iterate through the remaining values and compute derivative using central difference 
    for j in range(1, len(y) - 1):
        dy[j] = (y[j + 1] - y[j - 1])/(x[j + 1] - x[j - 1])

    #return derivative
    return(dy)

'''
Attempts to find the baseline fluorescence in a spectrum.
The algorithm is an implentation of SDT-RIA with some minor modifications, it essentially finds the baseline by iteratively smoothing the spectrum to remove the high frequency components.
An in depth explanation of the algorithm can be found here: https://doi.org/10.1364/OSAC.432785
Takes in the x and y data of a spectrum.
sgPasses (optional): sets the number of smoothing passes to apply to the second derivative.
trialWidth (optional): sets the initial width to use when attempting to fit the peak of the second derivative
Returns the baseline.
'''
def findBaseline(x, y, sgPasses = 3, trialWidth = 10):
    
    #calculate the second derivative of the spectrum
    dy = derivxy(x, y)
    d2y = derivxy(x, dy)
    
    #copy the second derivate, apply a Savisty-Gorlay filter to smooth, then remove border values where the data is unreliable
    d2ysg = d2y.copy()
    for i in range(sgPasses):
        d2ysg = sg(d2ysg, 25, 1)

    #find location of the minimum point
    #boundary values are unreliable, sometimes going off to extreme values, and so must be removed
    #the minimum here corresponds to the most prominant Raman peak
    d2ysg[:20] = 0
    d2ysg[-20:] = 0
    minimum = np.argmin(d2ysg)
    
    #anonymous function for the second derivative of a Lorentzian curve with parameters for height, linewidth, and peak centre
    d2Lorentzian = lambda x, h, w, p : h*(((128*((x - p)**2))/((w**4)*((((4*(x - p)**2)/(w**2)) + 1)**3))) - (8/((w**2)*((((4*(x - p)**2)/(w**2)) + 1)**2))))
    
    #fit a ideal second derivative Lorentzian to the most prominant peak using least squares
    fitParams = lsqFit(d2Lorentzian, x, d2y, [d2y[minimum], trialWidth, x[minimum]])[0]
    
    #create copy of the spectrum for iterative modification and store the amplitude of the most prominent peak
    yWorking = y.copy()
    yPeakAmplitude = y[minimum]
    dHeight = 0
    timeOut = 0
    
    #iteratively modify the spectrum to find baseline
    #the baseline is found when the differnce between the baseline and spectrum at the most prominent peak is the fitted height
    while(dHeight < fitParams[0]):
        
        #create a new trial baseline from the element-wise minimum of the previous trial baseline and original spectrum
        yWorking = np.minimum(yWorking, y)
        
        #smooth the spectrum using a Savisty-Gorlay filter, the wide filter window prevents the baseline from accidently including high frequency features
        yWorking = sg(yWorking, 101, 1)
        
        # calculate the height difference between the baseline and spectrum at the most prominent peak and how much this height difference has changed
        residual = dHeight - (yPeakAmplitude - yWorking[minimum])
        dHeight = yPeakAmplitude - yWorking[minimum]
        
        #to avoid cases of non-convergence increment a time out tracker each time there is no change in height difference
        if abs(residual) < 10e-3:
            timeOut += 1
            
            #if there has been no change after 100 consecutive smoothes terminate function and return a flat baseline
            if timeOut == 100:
                print('Baseline not converging, this usually indicates that there is not a strong baseline to the spectrum.')
                print('The function will return a flat baseline containing the minimum value of the spectrum.')
                return(np.full(len(y), min(y)))
        
        #if there is a change reset timeout counter
        else:
            timeOut = 0
    
    #return the final baseline
    return(yWorking)

'''
Automatically fits a spectrum using idealised Lorentzian peaks.
It is strongly reccomend to only pass spectra with their baseline removed to this function.
Takes the x and y data of a spectrum.
sgPasses (optional): sets the number of smoothing passes to apply to the second derivative.
trialWidth (optional): sets the initial width to use when attempting to fit the peak of the second derivative.
scaleFactor (optional): tells the program the x-axis resolution, used to approximately convert between array content and indices.
Returns a spectrum composed only of idealised Lorentzian peaks.
'''
def fitPeaks(x, y, sgPasses = 3, trialWidth = 10, scaleFactor = 0.525):

    
    dy = derivxy(x, y)
    d2y = derivxy(x, dy)
    d2ysg = d2y.copy()
    for i in range(sgPasses):
        d2ysg = sg(d2ysg, 25, 1)
    d2ysg[:50] = 0
    d2ysg[-50:] = 0
    
    peakLocs = [peak for peak in fp(-d2ysg, distance = 5*trialWidth, height = 1, prominence = 0.5)[0] if d2ysg[peak] < 0]
    
    print(peakLocs)
    
    lorentzian = lambda x, h, w, p : h/(1 + ((x - p)/(w/2))**2)
    d2Lorentzian = lambda x, h, w, p : h*(((128*((x - p)**2))/((w**4)*((((4*(x - p)**2)/(w**2)) + 1)**3))) - (8/((w**2)*((((4*(x - p)**2)/(w**2)) + 1)**2))))
    
    d2Params = []
    
    for peak in peakLocs:
        peakParams = [y[peak], trialWidth, x[peak]]
        peakBounds = ([0.9*y[peak], 0, x[peak] - trialWidth], [1.1*y[peak], 4*trialWidth, x[peak] + trialWidth])
        fitParams = lsqFit(d2Lorentzian, x, d2ysg, peakParams, bounds = peakBounds)[0]
        d2Params.append(fitParams)
    
    d2Params = sorted(d2Params, key = itemgetter(0), reverse = True)
    print(d2Params)
    
    '''
    refParams = []
    
    for peak in d2Params:
        tempLoc = peak[2]
        tempIndexMid = np.abs(x - tempLoc).argmin()
        tempIndexMin = np.abs(x - (tempLoc - peak[1])).argmin()
        tempIndexMax = np.abs(x - (tempLoc + peak[1])).argmin()
        tempx = x[tempIndexMin:tempIndexMax]
        tempy = y[tempIndexMin:tempIndexMax]
        if abs(peak[0] - y[tempIndexMid]) > 0.2*peak[0]:
            #refParams.append(lsqFit(lorentzian, x, y, [y[tempLoc], peak[1], peak[2]], maxfev = 10000)[0])
            reducedParams = lsqFit(lambda x, h, w : lorentzian(x, h, w, tempLoc), tempx, tempy, [y[tempIndexMid - tempIndexMin], peak[2]],  maxfev = 10000)[0]
            
            refParams.append([reducedParams[0], reducedParams[1], tempLoc])
        else:
            refParams.append(peak)
        
        #print(refParams)
    '''
    
    
    
    finalFits = []
    finalFit = np.zeros(len(x))
    for i in range(len(d2Params)):
        tempParams = d2Params[i]
        #tempFit = 100*d2Lorentzian(x, tempParams[0], tempParams[1], tempParams[2])
        tempFit = lorentzian(x, tempParams[0], tempParams[1], tempParams[2])
        finalFit += tempFit
        finalFits.append(tempFit)
        #finalFit = np.maximum(finalFit, lorentzian(x, temp[0], temp[1], temp[2]))
    
    
    
    
    return(100*d2ysg, finalFit, finalFits)

def coordination(path, isMap = False, xRange = 1, yRange = 1):
    
    if isMap:
        x, ys = importSpectrum(path, True, xRange, yRange)
        yProcessed = np.empty((yRange, xRange, len(x)))
        
        #for i in range(yRange):
            #for j in range(xRange):
                #print([i, j])
                #baseline = findBaseline(x, ys[i, j])
                #yProcessed[i, j] = ys[i, j] - baseline
        
        plot = go.FigureWidget([go.Scatter(x = x, y = ys[0][0], mode = 'lines')])
        
        red = [255, 0, 0]
        img = go.FigureWidget([go.Image(z = np.full((yRange, xRange, 3), red))])
        img.update_layout(modebar = {'remove' : ['toImage', 'zoom', 'pan', 'select', 'lasso2d', 'zoomIn', 'zoomOut', 'autoScale', 'resetScale']}, xaxis = {'fixedrange':True}, yaxis = {'fixedrange':True, 'scaleanchor':'x', 'scaleratio':1})
        
        out = Output()
        @out.capture(clear_output=True)

        def clickPixel(trace, points, selector):
            print(points)

            with plot.batch_update():
                #plot.data[0].y = yProcessed[points.ys[0], points.xs[0]]
                plot.data[0].y = ys[points.ys[0], points.xs[0]]
        
        img.data[0].on_click(clickPixel)

        dataPanel = VBox([img, out])
        dataMap = HBox([dataPanel, plot])
        return(dataMap)
        
        
    else:
        x, y = importSpectrum(path)
        baseline = findBaseline(x, y)
        #yFit, fit, finalFits = fitPeaks(x, y - baseline)
        createFig(x, [y, baseline, y - baseline])
        









#path = 'C:\\Users\\Owner\\Desktop\\White PET_01.txt'
#path = 'C:\\Users\\Owner\\Desktop\\PVC Pipe_02.txt'
#path = 'C:\\Users\\Owner\\Desktop\\Covid LFT Casing_01.txt'
#path = 'C:\\Users\\Owner\\Desktop\\Polypropylene_01.txt'
#path = 'C:\\Users\\Owner\\Desktop\\test.txt'
path = 'C:\\Users\\Owner\\Desktop\\Sample 2 Map.txt'

dataMap = coordination(path, True, 25, 25)

dataMap

    'data': [{'type': 'image',
              'uid': 'f8a5fe21-222…